# Observer Pattern

## Intent
Define a one-to-many dependency between objects so that when one object changes state, all its dependents are notified and updated automatically.

## Problem
You need to notify multiple objects when another object changes:
- Model-View updates (MVC architecture)
- Event handling systems
- Stock price monitoring
- Social media notifications
- Pub/Sub messaging

**Without Observer**: Objects are tightly coupled, hard to extend.

## When to Use
✅ **Use when:**
- Change in one object requires changing others
- Number of dependent objects is unknown
- Objects should be loosely coupled
- Broadcasting changes to multiple subscribers

❌ **Avoid when:**
- Simple one-to-one relationships
- Notifications are expensive
- Order of notifications matters critically

## Pattern Structure
```
┌─────────┐         ┌──────────┐
│ Subject │◄────────│ Observer │
├─────────┤         ├──────────┤
│ attach()│         │ update() │
│ detach()│         └──────────┘
│ notify()│               ▲
└─────────┘               │
     ▲           ┌────────┴────────┐
     │           │                 │
┌────┴─────┐  ┌──────┐      ┌──────────┐
│ Concrete │  │Observ│      │ Observer │
│ Subject  │  │er A  │      │    B     │
└──────────┘  └──────┘      └──────────┘
```

## Example 1: Stock Price Monitoring (Without Observer)

**Problem**: Tight coupling, hard to add new displays

In [ ]:
# WITHOUT Observer - Tightly coupled
class Stock:
    def __init__(self, symbol, price):
        self.symbol = symbol
        self.price = price
        self.display1 = None  # Tight coupling!
        self.display2 = None
    
    def set_price(self, price):
        self.price = price
        # Manually notify each display
        if self.display1:
            self.display1.update(self.symbol, self.price)
        if self.display2:
            self.display2.update(self.symbol, self.price)
        # What if we add display3? Need to modify this class!

class PriceDisplay:
    def update(self, symbol, price):
        print(f"Display: {symbol} is now ${price}")

# Usage - awkward setup
stock = Stock("AAPL", 150.0)
stock.display1 = PriceDisplay()
stock.display2 = PriceDisplay()
stock.set_price(155.0)  # Manually handles notifications

## Implementation: Observer Pattern

In [ ]:
from abc import ABC, abstractmethod
from typing import List

# Observer interface
class Observer(ABC):
    """Abstract observer interface."""
    
    @abstractmethod
    def update(self, subject: 'Subject') -> None:
        """Receive update from subject."""
        pass


# Subject interface
class Subject(ABC):
    """Abstract subject interface."""
    
    @abstractmethod
    def attach(self, observer: Observer) -> None:
        """Attach an observer."""
        pass
    
    @abstractmethod
    def detach(self, observer: Observer) -> None:
        """Detach an observer."""
        pass
    
    @abstractmethod
    def notify(self) -> None:
        """Notify all observers."""
        pass


# Concrete Subject
class Stock(Subject):
    """Stock that can be observed."""
    
    def __init__(self, symbol: str, price: float):
        self.symbol = symbol
        self._price = price
        self._observers: List[Observer] = []
    
    @property
    def price(self) -> float:
        return self._price
    
    @price.setter
    def price(self, value: float) -> None:
        """When price changes, notify observers."""
        if value != self._price:
            old_price = self._price
            self._price = value
            print(f"\n🔔 {self.symbol}: ${old_price} → ${value}")
            self.notify()
    
    def attach(self, observer: Observer) -> None:
        if observer not in self._observers:
            self._observers.append(observer)
            print(f"✓ Attached {observer.__class__.__name__} to {self.symbol}")
    
    def detach(self, observer: Observer) -> None:
        if observer in self._observers:
            self._observers.remove(observer)
            print(f"✗ Detached {observer.__class__.__name__} from {self.symbol}")
    
    def notify(self) -> None:
        """Notify all observers."""
        for observer in self._observers:
            observer.update(self)


# Concrete Observers
class PriceDisplay(Observer):
    """Displays current price."""
    
    def update(self, subject: Stock) -> None:
        print(f"  📊 Display: {subject.symbol} = ${subject.price}")


class EmailAlert(Observer):
    """Sends email if price crosses threshold."""
    
    def __init__(self, email: str, threshold: float):
        self.email = email
        self.threshold = threshold
    
    def update(self, subject: Stock) -> None:
        if subject.price > self.threshold:
            print(f"  📧 Email to {self.email}: {subject.symbol} above ${self.threshold}!")


class TradingBot(Observer):
    """Executes trades based on price."""
    
    def __init__(self, buy_at: float, sell_at: float):
        self.buy_at = buy_at
        self.sell_at = sell_at
    
    def update(self, subject: Stock) -> None:
        if subject.price <= self.buy_at:
            print(f"  🤖 Bot: BUY {subject.symbol} at ${subject.price}")
        elif subject.price >= self.sell_at:
            print(f"  🤖 Bot: SELL {subject.symbol} at ${subject.price}")


# Demo
print("=== Creating Stock and Observers ===")
apple = Stock("AAPL", 150.0)

display = PriceDisplay()
email = EmailAlert("trader@example.com", threshold=160.0)
bot = TradingBot(buy_at=145.0, sell_at=170.0)

apple.attach(display)
apple.attach(email)
apple.attach(bot)

print("\n=== Price Changes ===")
apple.price = 155.0  # Display shows
apple.price = 165.0  # Display + Email
apple.price = 140.0  # Display + Bot buys
apple.price = 175.0  # Display + Email + Bot sells

## Real-World Example: Weather Station

In [ ]:
class WeatherStation(Subject):
    """Weather station that broadcasts weather data."""
    
    def __init__(self):
        self._observers: List[Observer] = []
        self._temperature = 0.0
        self._humidity = 0.0
        self._pressure = 0.0
    
    def attach(self, observer: Observer) -> None:
        self._observers.append(observer)
    
    def detach(self, observer: Observer) -> None:
        self._observers.remove(observer)
    
    def notify(self) -> None:
        for observer in self._observers:
            observer.update(self)
    
    def set_measurements(self, temp: float, humidity: float, pressure: float):
        """Update weather measurements and notify."""
        self._temperature = temp
        self._humidity = humidity
        self._pressure = pressure
        print(f"\n🌤️  Weather Update: {temp}°C, {humidity}% humidity, {pressure} hPa")
        self.notify()
    
    @property
    def temperature(self):
        return self._temperature
    
    @property
    def humidity(self):
        return self._humidity
    
    @property
    def pressure(self):
        return self._pressure


class CurrentConditionsDisplay(Observer):
    """Shows current weather conditions."""
    
    def update(self, subject: WeatherStation) -> None:
        print(f"  📱 Current: {subject.temperature}°C, {subject.humidity}% humidity")


class StatisticsDisplay(Observer):
    """Shows statistics."""
    
    def __init__(self):
        self.temps = []
    
    def update(self, subject: WeatherStation) -> None:
        self.temps.append(subject.temperature)
        avg = sum(self.temps) / len(self.temps)
        print(f"  📊 Stats: Avg={avg:.1f}°C, Min={min(self.temps)}°C, Max={max(self.temps)}°C")


class ForecastDisplay(Observer):
    """Predicts weather based on pressure."""
    
    def __init__(self):
        self.last_pressure = 0
    
    def update(self, subject: WeatherStation) -> None:
        current = subject.pressure
        
        if current > self.last_pressure:
            forecast = "Improving weather"
        elif current < self.last_pressure:
            forecast = "Cooler, rainy weather"
        else:
            forecast = "More of the same"
        
        print(f"  🔮 Forecast: {forecast}")
        self.last_pressure = current


# Demo
print("=== Setting up Weather Station ===")
station = WeatherStation()

current_display = CurrentConditionsDisplay()
stats_display = StatisticsDisplay()
forecast_display = ForecastDisplay()

station.attach(current_display)
station.attach(stats_display)
station.attach(forecast_display)

print("\n=== Weather Updates ===")
station.set_measurements(25.0, 65.0, 1013.0)
station.set_measurements(27.0, 70.0, 1015.0)
station.set_measurements(23.0, 75.0, 1012.0)

## Python-Specific: Using Properties for Auto-Notification

In [ ]:
class ObservableProperty:
    """Descriptor that notifies observers on property changes."""
    
    def __init__(self, initial_value=None):
        self.value = initial_value
    
    def __set_name__(self, owner, name):
        self.name = f"_{name}"
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return getattr(instance, self.name, self.value)
    
    def __set__(self, instance, value):
        old_value = getattr(instance, self.name, None)
        setattr(instance, self.name, value)
        if hasattr(instance, 'notify'):
            instance.notify()


class User(Subject):
    """User with observable properties."""
    
    name = ObservableProperty()
    status = ObservableProperty()
    
    def __init__(self, name: str):
        self._observers = []
        self.name = name
        self.status = "offline"
    
    def attach(self, observer: Observer):
        self._observers.append(observer)
    
    def detach(self, observer: Observer):
        self._observers.remove(observer)
    
    def notify(self):
        for observer in self._observers:
            observer.update(self)


class StatusMonitor(Observer):
    """Monitors user status changes."""
    
    def update(self, subject: User):
        print(f"  👤 {subject.name} is now {subject.status}")


# Demo
user = User("Alice")
monitor = StatusMonitor()
user.attach(monitor)

print("Changing user status...")
user.status = "online"  # Auto-notifies!
user.status = "away"
user.status = "offline"

## Push vs Pull Models

### Push Model (Subject sends data)
Subject passes data to observers in update().

In [ ]:
# Push model - Subject pushes data
class PushObserver(ABC):
    @abstractmethod
    def update(self, data: dict) -> None:
        pass

class PushSubject:
    def __init__(self):
        self._observers = []
        self.data = {}
    
    def attach(self, observer):
        self._observers.append(observer)
    
    def notify(self):
        for observer in self._observers:
            observer.update(self.data)  # Push data

class PushDisplay(PushObserver):
    def update(self, data: dict):
        print(f"Push: Received {data}")

# Demo
subject = PushSubject()
subject.attach(PushDisplay())
subject.data = {'temp': 25}
subject.notify()

### Pull Model (Observer requests data)
Observer pulls data from subject when notified.

In [ ]:
# Pull model - Observer pulls data
class PullObserver(ABC):
    @abstractmethod
    def update(self, subject) -> None:
        pass

class PullSubject:
    def __init__(self):
        self._observers = []
        self.data = {}
    
    def attach(self, observer):
        self._observers.append(observer)
    
    def notify(self):
        for observer in self._observers:
            observer.update(self)  # Pass self
    
    def get_data(self):
        return self.data

class PullDisplay(PullObserver):
    def update(self, subject):
        data = subject.get_data()  # Pull data
        print(f"Pull: Retrieved {data}")

# Demo
subject = PullSubject()
subject.attach(PullDisplay())
subject.data = {'temp': 25}
subject.notify()

## Advantages & Disadvantages

### ✅ Advantages
1. **Loose coupling**: Subject and observers are independent
2. **Dynamic relationships**: Can add/remove observers at runtime
3. **Broadcast communication**: One-to-many updates
4. **Open/Closed Principle**: Add new observers without modifying subject

### ❌ Disadvantages
1. **Random notification order**: Can't guarantee sequence
2. **Memory leaks**: Forgotten observers keep references
3. **Performance**: Many observers = slow notifications
4. **Unexpected updates**: Hard to track cause of changes

## Common Pitfalls

### 1. Memory Leaks
```python
# Bad - observer not detached
observer = MyObserver()
subject.attach(observer)
# observer goes out of scope but subject keeps reference!

# Good - always detach
try:
    observer = MyObserver()
    subject.attach(observer)
    # use observer
finally:
    subject.detach(observer)
```

### 2. Notification Loops
```python
# Bad - infinite loop!
def update(self, subject):
    subject.state = new_value  # Triggers notify() again!

# Good - check before updating
def update(self, subject):
    if subject.state != new_value:
        subject.state = new_value
```

## Related Patterns

- **Mediator**: Mediator coordinates, Observer broadcasts
- **Singleton**: Subject often a singleton
- **Command**: Commands can be observers

## Python-Specific Alternatives

### 1. Using Python's signals (blinker library)
```python
from blinker import signal

price_changed = signal('price-changed')

@price_changed.connect
def on_price_change(sender, price):
    print(f"Price changed to {price}")

price_changed.send('AAPL', price=155.0)
```

### 2. Using Python's property with callbacks
```python
class Observable:
    def __init__(self):
        self._callbacks = []
    
    def observe(self, callback):
        self._callbacks.append(callback)
    
    def notify(self):
        for callback in self._callbacks:
            callback(self)
```

## Best Practices

1. **Always detach** observers when done
2. **Use weak references** for observers to avoid memory leaks
3. **Document notification guarantees** (order, thread-safety)
4. **Keep updates lightweight** - observers shouldn't do heavy work
5. **Consider async updates** for slow observers

## Summary

The Observer pattern enables:
- One-to-many relationships
- Loose coupling between subject and observers
- Dynamic subscription/unsubscription
- Event-driven architectures

Perfect for: UI updates, event systems, pub/sub messaging, reactive programming.